In [61]:
import os,sys
import numpy as np
import pandas as pd

# abspath = os.path.abspath(__file__)
# dname = os.path.dirname(abspath)
# os.chdir(dname) 

In [62]:
redistributed_primaries = pd.read_csv("processed/pct_post_redistribution.csv", index_col="DivisionNm")
redistributed_demographics = pd.read_csv("processed/pct_2025_demographic_facts.csv", index_col="DivisionNm")
candidates_2025 = pd.read_csv("processed/candidates.csv", index_col="DivisionNm")

In [63]:
prepoll_primaries = pd.DataFrame(index=candidates_2025.index, columns=candidates_2025.columns, dtype=float)

In [64]:
mapper = {"ALP":"ALP",'CLP':"LNP","GRN":"GRN", "IND":"IND", "JLN":"IND","KAP":"IND", "LNP":"LNP", "LP":"LNP", "NP":"LNP", "ON":"ONP", "TEAL":"TEAL", "XEN":"IND"}
for col in redistributed_primaries.columns:
    if col not in mapper:
        mapper[col] = "OTH"

redistributed_primaries_parties = redistributed_primaries.rename(mapper,axis=1).T.groupby(level=0).sum().T

In [65]:
for divisionNm, row in candidates_2025.iterrows():
    for partyAb, competing in row.items():
        if competing:
            prepoll_primaries.loc[divisionNm, partyAb] = redistributed_primaries_parties.loc[divisionNm, partyAb]

In [66]:
iidx_to_impute = list(zip(*np.where(((prepoll_primaries==0) | (prepoll_primaries.isna())) & (candidates_2025>0))))
idx_to_impute = [(prepoll_primaries.index[val[0]], prepoll_primaries.columns[val[1]]) for val in iidx_to_impute]

iidx_to_remove = list(zip(*np.where((redistributed_primaries_parties>0) & (candidates_2025==0))))
idx_to_remove = [(redistributed_primaries_parties.index[val[0]], redistributed_primaries_parties.columns[val[1]]) for val in iidx_to_remove]

In [75]:
prepoll_primaries.columns

Index(['ALP', 'GRN', 'IND', 'LNP', 'ONP', 'OTH', 'TEAL'], dtype='object')

In [81]:
M = pd.read_csv("M.csv",index_col="Unnamed: 0")
M = M[prepoll_primaries.columns].loc[prepoll_primaries.columns].to_numpy()

In [68]:
def eliminate_idx(idx,y:np.array,M:np.array, tol=1e-4, max_iter_sub=1000):
    if type(idx)==str:
        idx = ['ALP', 'GRN', 'IND', 'LNP', 'ONP', 'OTH', 'TEAL'].index(idx)
    if type(y)==pd.Series:
        yhat= y.to_numpy()
    else:
        yhat = y.copy()

    eliminated = (yhat==0)

    eliminated[idx] = True

    eliminating_idx = idx

    counter = 0
    while sum(yhat[eliminated]>0):
        flow = np.zeros_like(yhat)
        flow[eliminating_idx] = 1
        flow =  M @ flow

        yhat = yhat + flow*yhat[idx]
        yhat[eliminating_idx] = 0

        yhat[yhat<tol] = 0
        yhat = yhat / np.nansum(yhat)



        valid_indices = np.where((eliminated) & (yhat.reshape(-1) > 0))[0]
        if len(valid_indices)>0:
            eliminating_idx = valid_indices[0]
        else:
            break

        counter += 1
        if counter > max_iter_sub:
            break
    
    if type(y)==pd.Series:
        return pd.Series(yhat, index=y.index)
    return yhat

def eliminate_indices(indices,y:np.array,M:np.array, tol=1e-4, max_iter_sub=1000):
    yhat = y.copy()
    for idx in indices:
        yhat = eliminate_idx(idx,yhat,M,tol,max_iter_sub)
    return yhat

def eliminate_smallest(y:np.array,M:np.array, tol=1e-4, max_iter_sub=1000):
    idx = np.where(y==np.min(y[y>0]))[0][0]
    return eliminate_idx(idx,y,M,tol,max_iter_sub)

def TCP(y:np.array,M:np.array, tol=1e-4, max_iter_sub=1000):
    non_eliminated = sum(y>0)
    yhat = y.copy()
    while non_eliminated>2:
        yhat = eliminate_smallest(yhat,M,tol,max_iter_sub)
        non_eliminated = sum(yhat>0)
    return yhat

In [69]:
for (divisionNm, partyAb) in idx_to_remove:
    yhat = prepoll_primaries.loc[divisionNm]
    yhat[partyAb] = redistributed_primaries_parties.loc[divisionNm, partyAb]
    yhat = eliminate_idx(partyAb,yhat,M)
    prepoll_primaries.loc[divisionNm] = yhat

In [73]:
redistributed_primaries_parties.loc['Macnamara'].to_list()

[0.3130022518638911,
 0.3438556894988706,
 0.0245878269598813,
 0.2372835419309349,
 0.0129503514375905,
 0.06832033830883082,
 0.0]